# Документация 

In [10]:
import pandas as pd
import datetime
import numpy as np
from sklearn import preprocessing
from scipy.sparse import csr_matrix

## Описание входныхх данных

Для обучения используется только таблица events:

In [2]:
events = pd.read_csv('events.csv')
print(events.shape)
display(events)

(2756101, 5)


,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN
...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN
2756097,1438399813142,762376,view,115946,NaN
2756098,1438397820527,1251746,view,78144,NaN
2756099,1438398530703,1184451,view,283392,NaN


## Трансформации исходного датасета: 

Сортировка и перевод в формат datatime:

In [4]:
events = events.assign(date=pd.Series(datetime.datetime.fromtimestamp(i/1000).date() for i in events.timestamp))
events = events.sort_values('date').reset_index(drop=True)
events = events[['visitorid','itemid','event', 'date']]
events.head(5)

,visitorid,itemid,event,date
0,567743,42875,view,2015-05-03
1,190672,259357,view,2015-05-03
2,1127598,69533,view,2015-05-03
3,180270,379359,view,2015-05-03
4,425526,440984,view,2015-05-03


Фильтруем данные по времени событий:

In [5]:
start_date = '2015-5-3'
end_date = '2015-5-18'
fd = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date()
events = events[(events.date >= fd(start_date)) & (events.date <= fd(end_date))]

Сплитем датасет на обучение и тест для валидации:

In [7]:
split_point = int(np.round(events.shape[0]*0.8))
events_train = events.iloc[0:split_point]
events_test = events.iloc[split_point::]
events_test = events_test[(events_test['visitorid'].isin(events_train['visitorid']))
                          & (events_test['itemid'].isin(events_train['itemid']))]

Кодируем признаки, чтобы использовать их в качестве оценок:

In [9]:
id_cols=['visitorid','itemid']
trans_cat_train=dict()
trans_cat_test=dict()

for k in id_cols:
    cate_enc=preprocessing.LabelEncoder()
    trans_cat_train[k]=cate_enc.fit_transform(events_train[k].values)
    trans_cat_test[k]=cate_enc.transform(events_test[k].values)
    
ratings = dict()

cate_enc=preprocessing.LabelEncoder()
ratings['train'] = cate_enc.fit_transform(events_train.event)
ratings['test'] = cate_enc.transform(events_test.event)    

Создаём item-users sparse-матрицу:

In [12]:
n_users=len(np.unique(trans_cat_train['visitorid']))
n_items=len(np.unique(trans_cat_train['itemid']))

rate_matrix = dict()

#используем разреженные матрицы
rate_matrix['train'] = csr_matrix((ratings['train'],
                                   (trans_cat_train['visitorid'],
                                    trans_cat_train['itemid'])),
                                  shape=(n_users,n_items)
                                  )

rate_matrix['test'] = csr_matrix((ratings['test'],
                                  (trans_cat_test['visitorid'],
                                   trans_cat_test['itemid'])),
                                 shape=(n_users,n_items)
                                 )

## Параметры модели и метрики качества

В качестве модели использовалась ALS:

In [13]:
# модель
from implicit.als import AlternatingLeastSquares
# для оценки метрик
from implicit.evaluation import mean_average_precision_at_k
from implicit.evaluation import AUC_at_k

In [14]:
ALS_model = AlternatingLeastSquares(factors=10, random_state=42)
ALS_model.fit(rate_matrix['train'])

  0%|          | 0/15 [00:00<?, ?it/s]

Получение рекомендаций:

In [15]:
user_id = 1 # id пользователя
unique_items = np.unique(trans_cat_train['itemid']) # список айтемов

recomendations_ids, scores = ALS_model.recommend(user_id, rate_matrix['train'][user_id])
recomendations = unique_items[recomendations_ids]
print('Recomendations for user {}: {}'.format(user_id, recomendations))

Recomendations for user 1: [56602 52679 31132  9324 18259 36035 10178 52806  3972  6397]


Метрики:

In [17]:
map_at3 = mean_average_precision_at_k(ALS_model, rate_matrix['train'], rate_matrix['test'], K=3)
print('Mean Average Precision at 3: {:.5f}'.format(map_at3))
print('AUC_at_3:', AUC_at_k(ALS_model, rate_matrix['train'], rate_matrix['test'], K=3))

  0%|          | 0/3274 [00:00<?, ?it/s]

Mean Average Precision at 3: 0.00424


  0%|          | 0/3274 [00:00<?, ?it/s]

AUC_at_3: 0.5024250098884412


## Описание docker-образа и сервиса

Код для развертывания локального сервера с моделью:

In [ ]:
#from flask import Flask, request, jsonify
#import pickle
#import numpy as np

#with open(r'ALS_model.pkl', 'rb') as pkl_file1: 
#    model = pickle.load(pkl_file1)

#with open(r'unique_items.pkl', 'rb') as pkl_file2: 
#    unique_items = pickle.load(pkl_file2)
    
#with open(r'rate_matrix.pkl', 'rb') as pkl_file3: 
#    rate_matrix = pickle.load(pkl_file3)    
    
#app = Flask(__name__)

#@app.route('/recommendation')
#def hello_func():
#    user = int(request.args.get('name'))
#    recomendations_ids, scores = model.recommend(user, rate_matrix[user])
#    recomendations = unique_items[recomendations_ids]
#    return f'recommendation for user {user}:{recomendations[:3]}!'


#if __name__ == '__main__':
#    app.run(host='0.0.0.0', port=5000)

Docker-образ собирается из приложенного Dockerfile(docker build -t diplom .). В файле requirements указаны необходимые библиотеки. В папке app лежат файл сервера, сохранённая модель и необходимые матрицы в формате pkl.
Организация файлов в директории:
 
  
diplom

  ├─app

     ├─ALS_model.pkl

     ├─rate_matrix.pkl

     ├─unique_items.pkl

     └─server.py

  └─Dockerfile
  
  └─requirements.txt
  
Контейнер запускается из командной строки: docker run -it --rm --name=diplom -p=5000:5000 diplom
Далее по адресу http://127.0.0.1:5000/recommendation?user_id=35 можно получить рекомендации трёх товаров для для указанного user_id(указывается как /recommendation?user_id=номер_юзера)

Файл server.py можно запустить отдельно от docker-образа и сразу получить рекомендации на сервере.

Запуск образа:

![picture_of_bird](1.PNG)

Пример получения рекомендаций:

![picture_of_bird](2.PNG)
![picture_of_bird](3.PNG)